# Implementing the Hungarian Method

<b>Goal:</b> Implement the Hungarian Method.

Implement the Hungarian method for finding maximum weight matchings in weighted bipartite graphs. You may use the function `networkx.algorithms.bipartite.matching.maximum_matching` for finding a maximum cardinality matching in a bipartite graph, use the function `min_vertex_cover()` for finding a minimum cardinality vertex cover in a bipartite graph (following Theorem 3.9 from the script), or implement the whole algorithm from scratch.

## Implementation 

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

%matplotlib inline

In [ ]:
def min_vertex_cover(G, M):
    '''Computes the minimum cardinality vertex cover corresponding to the given maximum cardinality 
       matching following Theorem 3.9 in the script.
    
    Args:
        G: A bipartite networkx graph.
        M: A maximum cardinality matching to start from (provided as a set of pairs of vertices).
    
    Returns:
        A list of vertices in a minimum cardinality vertex cover.
    '''
    
    # Create default vertex and edge properties:
    # For a vertex, 
    #  - 'matched' indicates whether a vertex is matched (value 1) or unmatched (value 0),
    #  - 'part' indicates whether a vertex is in X (value 1) or Y (value 2),
    #  - 'L' indicates whether a vertex is marked, i.e., in L (value 1), or not (value 0).
    for v in G.nodes:
        G.nodes[v]['matched'] = G.nodes[v]['part'] = G.nodes[v]['L'] = 0
    # For an edge,
    #  - 'm' indicates whether an edges is in the matching (value 1) or not (value 0)    
    for u, w in G.edges:
        G[u][w]['m'] = G[w][u]['m'] = 0
        
    # Mark edges in M
    for u, w in M:
        G[u][w]['m'] = G[w][u]['m'] = 1
        G.nodes[u]['matched'] = G.nodes[w]['matched'] = 1
    
    # Partition G
    for v in G.nodes:
        if G.nodes[v]['part'] > 0:
            continue
        G.nodes[v]['part'] = 1
        for u, w in nx.bfs_edges(G, v):
            G.nodes[w]['part'] = G.nodes[u]['part'] % 2 + 1
    
    # Create the auxiliary directed graph
    D = nx.DiGraph()
    D.add_nodes_from(G.nodes)
    for u, w in G.edges:
        if G[u][w]['m'] == 0:
            if G.nodes[u]['part'] == 1:
                D.add_edge(u, w)
            else:
                D.add_edge(w, u)
        else:
            if G.nodes[u]['part'] == 1:
                D.add_edge(w, u)
            else:
                D.add_edge(u, w)
    
    # Mark vertices in L
    for v in G.nodes:
        if G.nodes[v]['part'] == 2 or G.nodes[v]['matched'] == 1 or G.nodes[v]['L'] == 1:
            continue
        G.nodes[v]['L'] = 1
        for u, w in nx.bfs_edges(D, v):
            G.nodes[w]['L'] = 1

    # Return S
    return [v for v in G.nodes
              if (G.nodes[v]['part'] == 1 and G.nodes[v]['L'] == 0) or
                 (G.nodes[v]['part'] == 2 and G.nodes[v]['L'] == 1)]

In [ ]:
def hungarian_method(G):
    '''Finds a minimum weight perfect matching in G using Hungarian Method.
    
    Args:
        G: A weighted biparitite networkx graph.
    
    Returns:
        If a minimum weight perfect matching does not exist, returns None.
        Otherwise, returns the matching in a dict where matching[i] == j if i is
        matched to j.
    '''
    # Write your implementation here

---

## Testing

Note that the `networkx` method `networkx.algorithms.bipartite.matching.minimum_weight_full_matching( )` (used in the testing function) requires the `SciPy` library to be installed. This dependency is installed automatically if you are using a recent version of the anaconda3 bundle. In case you already had an older version installed, you might have to manually upgrade `networkx` and/or instal `SciPy` using the commands

- `conda update networkx` and
- `conda install scipy`.

If you encounter problems, compare to how we installed the `python-mip` library when setting up the environment!

As always, also feel free to visualize at least the output of the Hungarian Method by implementing functionality to plot the input graph, for example with the maximum weight matching highlighted.

In [ ]:
def edge_list_weight(G, edge_list):
    '''Computes the weight of a list of edges in a graph.
    
    Args:
        G: A weighted networkx graph.
        edge_list: A list of edges of G.
    
    Returns:
        The weight of the edge list.
    '''
    return sum(G[i][j]['weight'] for i, j in edge_list)


def test_hungarian_method(G):
    '''Checks if hungarian_method() works correctly on a given instance.
    
    Args:
        G: A weighted bipartite netowrkx graph.
    
    Returns:
        A tuple consisting of three elements. The first element is True if 
        hungarian_method() returned a minimum weight perfect matching and 
        False otherwise. The second element is the cardinality of the edge
        set returned by hungarian_method() and the third element is the weight
        of this edge set.
    '''
    # Matcihng obtained with hungarian_method()
    M1 = hungarian_method(G)
    
    if M1 is None:
        validity = True
        cardinality = 0
        weight = 0
    else:
        validity = nx.is_matching(G, M1)
        cardinality = len(M1.keys()) / 2
        weight = edge_list_weight(G, M1.items()) / 2

    try:
        # Matching obtained with networkx
        M2 = nx.algorithms.bipartite.matching.minimum_weight_full_matching(G)
        validity = validity and 2 * cardinality == len(M2.keys()) and 2 * weight == edge_list_weight(G, M2.items())
    except ValueError:
        # No full matching exists
        validity = validity and M1 is None
    
    return validity, cardinality, weight

In [ ]:
# Instance 1
# Observe that this is the example from the script (chapter "The Hungarian Method by example"), so you can check intermediate steps if you want!
G1 = nx.complete_bipartite_graph(4, 4)

G1_weights = [[2, 6, 7, 4], [0, 7, 10, 7], [2, 8, 9, 7], [0, 1, 7, 0]]
G1.add_weighted_edges_from([(i, 4 + j, G1_weights[i][j]) for i in range(4) for j in range(4)])

print(test_hungarian_method(G1))
# Expected output: (True, 4.0, 14.0)

In [ ]:
# Instance 2
G2 = nx.complete_bipartite_graph(3, 3)

G2_weights = [[1, 2, 3], [3, 3, 3], [3, 3, 2]]
G2.add_weighted_edges_from([(i, 3 + j, G2_weights[i][j]) for i in range(3) for j in range(3)])

print(test_hungarian_method(G2))
# Expected output: (True, 3.0, 6.0)

In [ ]:
# Instance 3
G3 = nx.complete_bipartite_graph(4, 4)

G3_weights = [[1, 1, 1, 2], [3, 2, 4, 1], [4, 4, 2, 4], [2, 3, 3, 3, 3]]
G3.add_weighted_edges_from([(i, 4 + j, G3_weights[i][j]) for i in range(4) for j in range(4)])

print(test_hungarian_method(G3))
# Expected output: (True, 4.0, 6.0)

In [ ]:
# Instance 4
G4 = nx.complete_bipartite_graph(4, 4)

G4_weights = [[5, 7, 2, 0], [1, 9, 10, 4], [7, 8, 1, 11], [0, 2, 4, 5]]
G4.add_weighted_edges_from([(i, 4 + j, G4_weights[i][j]) for i in range(4) for j in range(4)])

print(test_hungarian_method(G4))
# Expected output: (True, 4.0, 4.0)

In [ ]:
# Instance 5
G5 = nx.complete_bipartite_graph(5, 5)

G5_weights = [[1, 2, 7, 3, 0], [3, 4, 1, 0, 2], [2, 3, 6, 0, 0], [0, 6, 1, 1, 0], [2, 0, 0, 4, 5]]
G5.add_weighted_edges_from([(i, 5 + j, G5_weights[i][j]) for i in range(5) for j in range(5)])

print(test_hungarian_method(G5))
# Expected output: (True, 5.0, 1.0)

In [ ]:
# Instance 6
G6 = nx.complete_bipartite_graph(5, 5)

G6_weights = [[30, 18, 9, 39, 97], [30, 3, 56, 9, 3], [86, 94, 13, 31, 34], [24, 72, 59, 30, 4], [10, 87, 25, 57, 29]]
G6.add_weighted_edges_from([(i, 5 + j, G6_weights[i][j]) for i in range(5) for j in range(5)])

print(test_hungarian_method(G6))
# Expected output: (True, 5.0, 54.0)

In [ ]:
# Instance 7
# (infeasible)
G7 = nx.Graph()
G7.add_weighted_edges_from([(1, 4, 1), (1, 5, 2), (1, 6, 4), (2, 4, 3), (3, 4, 7)])

print(test_hungarian_method(G7))
# Expected output: (True, 0, 0)